In [0]:
valuesA = [('Pirate', 1), ('Monkey', 2), ('Ninja', 3), ('Spaghetti', 4)]
TableA = spark.createDataFrame(valuesA, ['name', 'id'])

valuesB = [('Ritabaga', 1), ('Pirate', 2), ('Ninja', 3), ('Darth Vader', 4)]
TableB = spark.createDataFrame(valuesB, ['name', 'id'])

TableA.show()
TableB.show()

+---------+---+
 name| id|
+---------+---+
 Pirate| 1|
 Monkey| 2|
 Ninja| 3|
Spaghetti| 4|
+---------+---+

+-----------+---+
 name| id|
+-----------+---+
 Ritabaga| 1|
 Pirate| 2|
 Ninja| 3|
Darth Vader| 4|
+-----------+---+

In [0]:
ta = TableA.alias('ta')
tb = TableB.alias('tb')

inner_join = ta.join(tb, ta.name == tb.name)
inner_join.show()

+------+---+------+---+
 name| id| name| id|
+------+---+------+---+
 Ninja| 3| Ninja| 3|
Pirate| 1|Pirate| 2|
+------+---+------+---+

In [0]:
left_join = ta.join(tb, ta.name == tb.name, how='left')
left_join.show()

+---------+---+------+----+
 name| id| name| id|
+---------+---+------+----+
Spaghetti| 4| null|null|
 Ninja| 3| Ninja| 3|
 Pirate| 1|Pirate| 2|
 Monkey| 2| null|null|
+---------+---+------+----+

In [0]:
from pyspark.sql.functions import col
left_join = ta.join(tb, ta.name == tb.name, how='left')
left_join.filter(col('tb.name').isNull()).show()

+---------+---+----+----+
 name| id|name| id|
+---------+---+----+----+
Spaghetti| 4|null|null|
 Monkey| 2|null|null|
+---------+---+----+----+

In [0]:
right_join = ta.join(tb, ta.name == tb.name, how='right')
right_join.show()

+------+----+-----------+---+
 name| id| name| id|
+------+----+-----------+---+
 Ninja| 3| Ninja| 3|
Pirate| 1| Pirate| 2|
 null|null| Ritabaga| 1|
 null|null|Darth Vader| 4|
+------+----+-----------+---+

In [0]:
full_outer_join = ta.join(tb, ta.name == tb.name, how='full')
full_outer_join.show()

+---------+----+-----------+----+
 name| id| name| id|
+---------+----+-----------+----+
Spaghetti| 4| null|null|
 Ninja| 3| Ninja| 3|
 Pirate| 1| Pirate| 2|
 Monkey| 2| null|null|
 null|null| Ritabaga| 1|
 null|null|Darth Vader| 4|
+---------+----+-----------+----+

In [0]:
llist = [('bob', '2015-01-13', 4), ('alice', '2015-04-13', 10)]
left = spark.createDataFrame(llist, ['name', 'date', 'duration'])
right = spark.createDataFrame([('alice', 100), ('bob', 23)], ['name', 'upload'])

df = left.join(right, left.name == right.name)
display(df)

name,date,duration,name,upload
alice,2015-04-13,10,alice,100
bob,2015-01-13,4,bob,23


In [0]:
df = left.join(right, ['name'])
display(df)

name,date,duration,upload
alice,2015-04-13,10,100
bob,2015-01-13,4,23


In [0]:
peopleDF = spark.read.parquet("/FileStore/tables/people-10m.parquet")
peopleDF.show(1)
ssaDF = spark.read.parquet("/FileStore/tables/people-10m.parquet")
ssaDF.show(1)

+---+---------+----------+----------+------+-------------------+-----------+------+
 id|firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---+---------+----------+----------+------+-------------------+-----------+------+
 1| Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
+---+---------+----------+----------+------+-------------------+-----------+------+
only showing top 1 row

+---+---------+----------+----------+------+-------------------+-----------+------+
 id|firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---+---------+----------+----------+------+-------------------+-----------+------+
 1| Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
+---+---------+----------+----------+------+-------------------+-----------+------+
only showing top 1 row

In [0]:
from pyspark.sql.functions import avg
avgSalaryDF = peopleDF.select(avg("salary").alias("averageSalary"))
avgSalaryDF.show()

+-------------+
averageSalary|
+-------------+
72633.0076033|
+-------------+

In [0]:
from pyspark.sql.functions import round
roundAvgSalaryDF = avgSalaryDF.select(round("averageSalary").alias("roundedAverageSalary"))
roundAvgSalaryDF.show()

+--------------------+
roundedAverageSalary|
+--------------------+
 72633.0|
+--------------------+

In [0]:
from pyspark.sql.functions import min, max
salaryDF = peopleDF.select(max("salary").alias("max"), min("salary").alias("min"), round(avg("salary")).alias("averageSalary"))
salaryDF.show()

+------+------+-------------+
 max| min|averageSalary|
+------+------+-------------+
180841|-26884| 72633.0|
+------+------+-------------+

In [0]:
print(peopleDF.count())
peopleDFDistinctNamesDF = peopleDF.select("firstName").distinct()
print(peopleDFDistinctNamesDF.count())
peopleDFDistinctNamesDF.show(1)

10000000
5113
+---------+
firstName|
+---------+
 Alayna|
+---------+
only showing top 1 row

In [0]:
print(ssaDF.count())
ssaDFDistinctNamesDF = ssaDF.select("firstName").distinct()
print(ssaDFDistinctNamesDF.count())
ssaDFDistinctNamesDF.show(1)

10000000
5113
+---------+
firstName|
+---------+
 Alayna|
+---------+
only showing top 1 row

In [0]:
ssaDFDistinctNamesDF = ssaDF.select("firstName")\
                            .withColumnRenamed("firstName", "ssaFirstName").distinct()
print(ssaDFDistinctNamesDF.count())
ssaDFDistinctNamesDF.show(3)

5113
+------------+
ssaFirstName|
+------------+
 Alayna|
 Melaine|
 Faye|
+------------+
only showing top 3 rows

In [0]:
joinedDF = peopleDFDistinctNamesDF.join(ssaDFDistinctNamesDF, \
                                       peopleDFDistinctNamesDF.firstName == ssaDFDistinctNamesDF.ssaFirstName)
joinedDF.count()

Out[19]: 5113

In [0]:
from pyspark.sql.functions import col
joinedDF = peopleDFDistinctNamesDF.join(ssaDFDistinctNamesDF, \
                                       col("firstName") == col("ssaFirstName"))
joinedDF.count()

Out[20]: 5113

In [0]:
# upper나 abs 함수의 경우 반드시 col()이 인자로 들어와야 한다
from pyspark.sql.functions import upper
peopleDF.select(upper(col("firstName"))).show(3)

+----------------+
upper(firstName)|
+----------------+
 PENNIE|
 AN|
 QUYEN|
+----------------+
only showing top 3 rows

In [0]:
from pyspark.sql.functions import abs
peopleWithFixedSalariesDF = peopleDF.select("firstName", "middleName", \
                                             "lastName", "gender", "birthDate", "ssn", abs(col("salary")).alias("salary"))
peopleDFDistinctNamesDF.count()

Out[22]: 5113

In [0]:
peopleWithFixedSalariesDF.show(3)

+---------+----------+----------+------+-------------------+-----------+------+
firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---------+----------+----------+------+-------------------+-----------+------+
 Pennie| Carry|Hirschmann| F|1955-07-02 04:00:00|981-43-9345| 56172|
 An| Amira| Cowper| F|1992-02-08 05:00:00|978-97-8086| 40203|
 Quyen| Marlen| Dome| F|1970-10-11 04:00:00|957-57-8246| 53417|
+---------+----------+----------+------+-------------------+-----------+------+
only showing top 3 rows

In [0]:
peopleWithFixedSalariesSortedDF = peopleWithFixedSalariesDF.select("*")\
                                  .orderBy("salary").limit(20)

In [0]:
peopleWithFixedSalariesSortedDF.show(4)

+---------+----------+--------+------+-------------------+-----------+------+
firstName|middleName|lastName|gender| birthDate| ssn|salary|
+---------+----------+--------+------+-------------------+-----------+------+
 Janene| Lili| Prinn| F|1986-04-06 05:00:00|923-50-6804| 2|
 Brook| Winifred| Durnell| F|1999-09-07 04:00:00|989-18-7019| 3|
 Garret| Garrett| Ashling| M|1959-10-19 04:00:00|918-39-6461| 4|
 Doloris| Domenica| Matic| F|1984-08-03 04:00:00|928-53-3688| 5|
+---------+----------+--------+------+-------------------+-----------+------+
only showing top 4 rows

In [0]:
peopleWithSalariesSortedDF = peopleWithFixedSalariesDF.select("*").\
                            orderBy("salary", ascending=False).limit(20).show(4)

+---------+----------+---------+------+-------------------+-----------+------+
firstName|middleName| lastName|gender| birthDate| ssn|salary|
+---------+----------+---------+------+-------------------+-----------+------+
 Belinda| Talia| Jessard| F|1955-05-15 04:00:00|966-31-6469|180841|
 Shena| Patty| Grinston| F|1989-09-02 04:00:00|980-60-8702|173969|
 Courtney| Kip| Liell| M|1986-12-06 05:00:00|981-68-2592|170562|
 Clarence| Allan|MacDuffie| M|1967-07-02 04:00:00|924-59-6873|170371|
+---------+----------+---------+------+-------------------+-----------+------+
only showing top 4 rows